# 遗传算法解决背包问题

In [1]:
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5 # 基因编码位数
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

# To assure reproductibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

# Create the item dictionary: item name is an integer, and value is
# a (weight, value) 2-uple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0  # Ensure overweighted bags are dominated
    return weight, value,


def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)  # Used in order to keep type
    ind1 &= ind2  # Intersection (inplace)
    ind2 ^= temp  # Symmetric Difference (inplace)
    return ind1, ind2


def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:  # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,


toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)


def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)

    return pop, stats, hof


if __name__ == "__main__":
    pop, stats, hof = main()
    print("最佳装包为(最佳个体) :",hof[-1])
    print(len(pop))
    print(len(hof))
    print("最佳装包时的重量与价值(最佳适应度) :",evalKnapsack(hof[-1]))

gen	nevals	avg                          	std                        	min                        	max                          
0  	50    	[  22.78        210.00877715]	[  5.88316241  71.09309649]	[ 10.          49.69958685]	[  38.          345.35491309]
1  	87    	[   9.96        145.20790666]	[  10.61312395  139.1868852 ]	[ 0.  0.]                  	[  45.          414.48478381]
2  	91    	[  4.26        78.58058033]  	[   8.44703498  138.29349751]	[ 0.  0.]                  	[  32.          423.36695161]
3  	90    	[  3.76        79.73012141]  	[   6.92693294  131.91906459]	[ 0.  0.]                  	[  22.          430.68948071]
4  	92    	[  4.14        89.56204765]  	[   6.96565862  138.08721534]	[ 0.  0.]                  	[  22.          430.68948071]
5  	87    	[   4.82        108.69011028]	[   7.42344933  144.62207924]	[ 0.  0.]                  	[  22.          430.68948071]
6  	84    	[   6.1       132.245644]    	[   8.25893456  153.8768951 ]	[ 0.  0.]                  	[ 